In [1]:
# is a good pratice only import what you need in the notebook , ill refactor this later in these notebooks :)

import sys
sys.path.append(r'C:\Users\bield\Documents\projsData\proj-1\utils')
from utils import (createSparkSesion,
                  takePandas)
from pyspark.sql import (functions as F,
                        Window)
import matplotlib.pyplot as plt



In [22]:
sys.path.append(r'C:\Users\bield\Documents\projsData\proj-1\utils')

In [8]:
spark = createSparkSesion()

In [ ]:
prefix = r"../../datasets/download/Atividades/"


In [30]:
df_clients     = spark.read.parquet(f"{prefix}/Clientes.parquet")
df_sales_itens = spark.read.parquet(f"{prefix}/ItensVendas.parquet")
df_products    = spark.read.parquet(f"{prefix}/Produtos.parquet")
df_sales       = spark.read.parquet(f"{prefix}/Vendas.parquet")
df_seller      = spark.read.parquet(f"{prefix}/Vendedores.parquet")

In [31]:
df_clients.takePandas(2)

,ClienteID,Cliente,Estado,Genero,Status
0,1,Adelina Buenaventura,RJ,M,Silver
1,2,Adelino Gago,RJ,M,Silver


In [33]:
df_sales_itens.takePandas(2)

,ProdutoID,VendasID,Quantidade,ValorUnitario,ValorTotal,Desconto,TotalComDesconto
0,2,400,2,9201.0,18402.0,"6256,68",12145.32
1,2,385,2,9201.0,18402.0,"5704,62",12697.38


In [34]:

df_products.takePandas(2)

,ProdutoID,Produto,Preco
0,1,Bicicleta Aro 29 Mountain Bike Endorphine 6.3 ...,"8.852,00"
1,2,Bicicleta Altools Stroll Aro 26 Freio À Disco ...,"9.201,00"


In [35]:
df_sales.takePandas(2)

,VendasID,VendedorID,ClienteID,Data,Total
0,1,1,91,1/1/2019,8053.6
1,2,6,185,1/1/2020,150.4


In [36]:
df_seller.takePandas(2)

,VendedorID,Vendedor
0,1,Armando Lago
1,2,Capitolino Bahía


In [ ]:
#my objective here is not to do a profund business analysis , just a few insights to build a pyspark code , and exemplify my abilities in build a ETL , desnormazile tables etc...

In [ ]:
#to do

#sales man metrics
#with seller give more discounts
#best sellers by month >by amount , by sales
#objective  > identify best emplooyes ,for maybe a bonus , and find a possible corelation between give discounts, and be the best seller



#clients metrics 
#top client , by amount , by recurrence
#maybe detect with client we could offer discounts , or send marketing campaings etc...




#store metrics:
#month over month
#month distribuition , by amount , by untary sales
#understando general store metrics



#product metrics:
#top product , by unitys sold, by amount
#understand which product we sell more , for some kind of storage managment , tendencies etc..



<h1>Sellers metrics 

a maneger could use this etl to track sellers performance , for example

In [ ]:
df_sellers_metrics  = (
                        df_sales.join(df_seller,on="VendedorID",how="left")
                                .join(df_sales_itens,on="VendasID",how="left")
                                .select(
                                        F.expr("VendasID as sale_id"),
                                        F.expr("Vendedor as seller_name"),
                                        F.expr("Total as total_sale_amount"),
                                        F.expr("ProdutoId as product_id"),
                                        F.expr("TotalComDesconto as item_final_price_with_discont"),
                                        F.expr("Desconto as discont"),
                                        F.expr("Data as date")
                                        )
                                .withColumn("discont",
                                                        F.when(F.col("discont").rlike(r"-"),0.00)
                                                         .otherwise(
                                                                        F.regexp_replace(F.col("discont"),r",",r".")
                                                                        .cast("double")) # tratando os nulos
                                            )
                                .withColumn("date",F.to_date(F.to_timestamp(F.col("date"), "d/M/yyyy"))) # cast para time stamp facilita funcoes de calculo e operacoes com datas
                        )


In [103]:
df_sellers_metrics.takePandas(2)

,sale_id,seller_name,total_sale_amount,product_id,item_final_price_with_discont,discont,date
0,1,Armando Lago,8053.6,2,7820.85,0.00,2019-01-01
1,1,Armando Lago,8053.6,9,96.77,0.98,2019-01-01


In [98]:
df_sellers_metrics.printSchema()

root
 |-- sale_id: long (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- total_sale_amount: double (nullable = true)
 |-- product_id: long (nullable = true)
 |-- item_final_price_with_discont: double (nullable = true)
 |-- discont: double (nullable = true)
 |-- date: string (nullable = true)



In [46]:
df_sales_itens.filter(F.col("VendasID")==1).takePandas(20)

,ProdutoID,VendasID,Quantidade,ValorUnitario,ValorTotal,Desconto,TotalComDesconto
0,10,1,1,135.00,135.00,"1,35",133.65
1,9,1,1,97.75,97.75,"0,98",96.77
2,2,1,1,7820.85,7820.85,-,7820.85


In [ ]:
df_sales_itens.prion